In [1]:
import os
import numpy as np
import pandas as pd

In [19]:
dir_path = "./model_with_diff_p_dec_enc_final_dense_pos/enc_train_only_2/"
file_path = dir_path + "train_hist.txt"

In [20]:
data_dict = {"train" : [],
              "val" : []}
cols = []
with open(file_path, "r") as f:
    all_data = f.readlines()
    for cur_line in all_data:
        if cur_line.endswith("train metrics:\n"):
            dtype = "train"
        elif cur_line.endswith("val metrics:\n"):
            dtype = "val"
        elif cur_line.startswith("kl_scale"):
            single_metrics = cur_line.strip().split(",")
            single_metrics = single_metrics[:-1]
            metric_vals = [float(cur_metric.strip().split(":")[1]) for cur_metric in single_metrics]
            cols = [cur_metric.strip().split(":")[0] for cur_metric in single_metrics]
            data_dict[dtype].append(np.array(metric_vals))

In [23]:
train_data = pd.DataFrame(data_dict["train"], columns = cols)
val_data = pd.DataFrame(data_dict["val"], columns = ["val_" + cur_col for cur_col in cols])
empty_row = pd.DataFrame(dict(zip(cols + ["val_" + cur_col for cur_col in cols], [[1.0]] * len(cols) * 2)))
full_df = pd.concat([train_data, val_data], axis = 1)
full_df = pd.concat([empty_row, full_df])
full_df.to_csv(dir_path + "train_hist.csv", index = True)